In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import logging
import os
import xml.etree.ElementTree as ET


In [2]:
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.DEBUG
)

for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [3]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_4/'

In [4]:
sample_dir = base_dir + 'data/samples/wgamma_sys/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma_sys/'
log_dir = base_dir + 'logs/wgamma_sys/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'

## Parse LHE file

In [5]:
filename = mg_process_dir + "/Events/run_01/unweighted_events.lhe"

In [7]:
tree = ET.parse(filename)
root = tree.getroot()

In [27]:
weight_groups = root.findall("header")[0].findall("initrwgt")[0].findall("weightgroup")

In [45]:
wg = weight_groups[1]

In [47]:
wg.attrib['name']

'CT10nlo'

In [53]:
w = wg.findall("weight")[0]

In [55]:
w.attrib

{'id': '45', 'MUR': '1.0', 'MUF': '1.0', 'PDF': '11000'}